In [1]:
import pandas as pd
import preprocess as pp
import datetime
import numpy as np
import eda
import sklearn
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [2]:
features_jan_2015 = pp.read_npy1('/u/project/cratsch/tescala/month_split_right/features_jan_2015.npy', features=True)
targets_jan_2015 = pp.read_npy1('/u/project/cratsch/tescala/month_split_right/targets_jan_2015.npy', targets=True)

In [3]:
new_features = features_jan_2015.loc[:, 'datetime':'z1']
z_list = ['z2', 'z3', 'z4', 'z5', 'z6', 'z7', 'z8', 'z9', 'z10', 'z11', 'z12']
for z in z_list:
    new_features[z] = features_jan_2015[z]

In [4]:
# center the data around 0 and fill the NaN's with 0 
def normalize_and_fill(df):
    
    m = df.loc[:,'x1':'z12'].mean()
    std = df.loc[:,'x1':'z12'].std()
    df.loc[:,'x1':'z12'] = (df.loc[:,'x1':'z12']-m) / std

    normalized_df = df.fillna(0)
    
    return normalized_df

In [5]:
df = normalize_and_fill(new_features)
df.head()

,datetime,squ,weights,x1,x2,x3,x4,x5,x6,x7,...,z3,z4,z5,z6,z7,z8,z9,z10,z11,z12
0,2015-01-02 09:35:00-05:00,22447.0,0.000211,-0.201069,-0.371059,-0.941790,-0.010691,-2.301466,-0.410278,-0.600753,...,1.401329,0.142898,-1.017979,-0.905067,-0.298542,1.649684,2.143122,1.773952,-0.268730,-0.203372
1,2015-01-02 09:35:00-05:00,22449.0,0.000167,0.364256,0.657008,1.103500,-0.187467,-1.994127,-0.356053,-0.519593,...,1.908666,-0.073253,-1.001275,-0.891300,-0.388311,0.011305,2.143122,-1.708193,-0.059738,-0.369460
2,2015-01-02 09:35:00-05:00,22462.0,0.001675,-0.381891,-0.666556,-1.102231,0.060626,-1.592686,-0.314737,-0.451606,...,0.880989,0.186855,-0.851415,-0.686007,0.027355,0.048018,0.584323,1.296794,-0.547161,2.528332
3,2015-01-02 09:35:00-05:00,22463.0,0.001890,0.017349,0.044416,0.102547,-0.118019,0.751048,-0.040355,-0.054525,...,0.238973,0.012115,-1.237260,-1.207407,-0.945570,0.533145,-0.589655,-0.376421,-0.813299,2.791516
4,2015-01-02 09:35:00-05:00,22470.0,0.001376,-0.327322,-0.561240,-0.845025,0.029654,-1.216952,-0.281944,-0.407436,...,0.565167,0.060295,-0.968731,-0.849193,-0.330980,0.832817,-0.825024,-0.385166,0.688875,2.130414


In [6]:
new_features.loc[:,'x1':'z12'].describe()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,z3,z4,z5,z6,z7,z8,z9,z10,z11,z12
count,4.046901e+06,4.047454e+06,4.049253e+06,4.050261e+06,4.047837e+06,4.047889e+06,4.047889e+06,4.047889e+06,4.050261e+06,4.045859e+06,...,4.049211e+06,4.050261e+06,4.046901e+06,4.047454e+06,4.049253e+06,4.050261e+06,4.047837e+06,4.050261e+06,4.050349e+06,4.050786e+06
mean,2.137623e-07,4.927280e-08,-1.612001e-07,-1.360803e-07,9.849148e-08,-1.860522e-07,4.137173e-08,2.830574e-08,-4.431311e-07,-4.265755e-07,...,2.386140e-05,3.400570e-06,1.039134e-04,-9.869298e-04,-1.452162e-03,6.797952e-05,3.034934e-03,-4.897229e-07,1.399160e-03,-1.070328e-03
std,1.000146e+00,1.000413e+00,1.000527e+00,1.000201e+00,1.000167e+00,1.000071e+00,1.000347e+00,1.000061e+00,1.000069e+00,1.000282e+00,...,1.000189e+00,1.000316e+00,1.000300e+00,1.000667e+00,1.000316e+00,1.000128e+00,9.973215e-01,1.000294e+00,1.000691e+00,1.000390e+00
min,-4.981863e+00,-7.107987e+00,-8.408089e+00,-2.776002e+00,-3.077624e+00,-4.373597e+00,-4.693880e+00,-7.625864e+00,-2.500741e+00,-4.295129e+00,...,-2.542524e+00,-2.741985e+00,-1.272009e+00,-1.254048e+00,-1.078838e+00,-2.648764e+00,-1.163653e+00,-2.592191e+00,-8.709112e-01,-1.198144e+00
25%,-4.762659e-01,-4.466591e-01,-4.235763e-01,-6.467194e-01,-6.563225e-01,-4.913952e-01,-4.786575e-01,-4.303375e-01,-6.909981e-01,-4.840344e-01,...,-6.812554e-01,-6.480750e-01,-6.709874e-01,-6.753128e-01,-6.586421e-01,-6.724268e-01,-9.176862e-01,-6.848024e-01,-6.142589e-01,-7.669820e-01
50%,-4.677243e-03,-6.341441e-03,-3.797395e-03,-5.554817e-02,-2.316217e-02,-2.699883e-03,-4.902532e-03,-5.295215e-03,5.695150e-05,-5.032886e-03,...,-5.089985e-02,-4.946427e-02,-2.926165e-01,-2.918884e-01,-3.024835e-01,2.343663e-02,-2.368383e-01,-1.516813e-03,-2.942770e-01,-3.169156e-01
75%,4.756112e-01,4.410451e-01,4.190094e-01,5.754645e-01,6.657533e-01,4.920218e-01,4.763790e-01,4.241051e-01,6.953218e-01,4.873671e-01,...,6.202833e-01,5.683405e-01,3.310457e-01,3.364138e-01,3.052939e-01,6.800257e-01,7.893888e-01,6.964002e-01,2.263067e-01,4.965020e-01
max,4.675741e+00,6.267026e+00,8.229766e+00,3.279141e+00,3.227858e+00,4.220837e+00,5.099127e+00,7.768436e+00,2.469715e+00,3.789541e+00,...,3.618345e+00,3.474801e+00,5.671055e+00,6.663046e+00,8.849887e+00,2.424059e+00,2.143122e+00,2.528948e+00,8.054776e+00,3.377297e+00


In [7]:
features_jan_2015.loc[:,'x1':'z12'].describe()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,z9 x z15,z10,z10 x z13,z10 x z14,z10 x z15,z11,z11 x z13,z11 x z14,z11 x z15,z12
count,4.046901e+06,4.047454e+06,4.049253e+06,4.050261e+06,4.047837e+06,4.047889e+06,4.047889e+06,4.047889e+06,4.050261e+06,4.045859e+06,...,4.047312e+06,4.050261e+06,4.050261e+06,4.050261e+06,4.050261e+06,4.050349e+06,4.050349e+06,4.050349e+06,4.049824e+06,4.050786e+06
mean,6.455011e-05,1.533171e-05,1.462109e-06,-7.727414e-03,-2.580688e-04,1.091448e-05,4.293053e-06,3.014885e-06,-2.657422e-04,1.608966e-04,...,8.492442e-04,-1.371452e-04,6.755690e-06,1.162361e-05,-1.821224e-07,2.292255e-02,-1.001081e-03,-2.028914e-03,5.501342e-05,-9.653682e-01
std,7.611257e-03,4.111419e-03,1.856240e-03,1.054806e+00,4.715497e-01,5.401611e-03,3.693900e-03,1.942266e-03,4.821859e-02,1.247010e-02,...,7.928939e-04,4.821458e-02,4.623196e-02,4.474732e-02,1.195716e-04,2.614203e-02,3.328690e-02,3.215429e-02,6.519026e-05,7.674354e-01
min,-3.785369e-02,-2.920859e-02,-1.560597e-02,-2.935871e+00,-1.451511e+00,-2.361356e-02,-1.733443e-02,-1.480844e-02,-1.208480e-01,-5.339978e-02,...,0.000000e+00,-1.251186e-01,-2.050211e-01,-2.564983e-01,-6.628896e-04,1.551618e-04,-3.826953e-01,-2.609923e-01,3.841249e-07,-1.884866e+00
25%,-3.560432e-03,-1.821071e-03,-7.847972e-04,-6.898910e-01,-3.097468e-01,-2.643411e-03,-1.763820e-03,-8.328149e-04,-3.358470e-02,-5.875058e-03,...,1.731168e-04,-3.315461e-02,-2.121975e-02,-2.140522e-02,-7.670750e-05,6.864576e-03,-1.072802e-02,-1.287718e-02,1.579077e-05,-1.553977e+00
50%,2.895042e-05,-1.074062e-05,-5.586769e-06,-6.631996e-02,-1.118018e-02,-3.669231e-06,-1.381641e-05,-7.269829e-06,-2.629960e-04,9.813607e-05,...,6.486274e-04,-2.102777e-04,1.418101e-19,1.408422e-05,-4.816486e-07,1.522955e-02,-1.718123e-04,-2.143753e-03,3.546419e-05,-1.208580e+00
75%,3.684549e-03,1.828653e-03,7.792441e-04,5.992761e-01,3.136778e-01,2.668625e-03,1.763989e-03,8.267396e-04,3.326169e-02,6.238411e-03,...,1.363874e-03,3.343950e-02,2.122325e-02,2.140377e-02,7.742870e-05,2.883866e-02,9.256415e-03,6.182018e-03,6.852802e-05,-5.843350e-01
max,3.565282e-02,2.578171e-02,1.527788e-02,3.451131e+00,1.521837e+00,2.281023e-02,1.883996e-02,1.509138e-02,1.188204e-01,4.741683e-02,...,6.359279e-03,1.217950e-01,2.106156e-01,2.496849e-01,6.154518e-04,2.334907e-01,3.722684e-01,4.649326e-01,1.110166e-03,1.626489e+00


## Set up for correlation matrix
Measure of model similarity

In [ ]:
# array of predictions for 3 models
# ols lin reg predict
# v1
# v2
# v3
predictions = ([ , , , ],
              [ , , , ],
            [ , , , ])

In [ ]:
correlation_matrix = np.corrcoef(predictions)